In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('./source/etl'))

from pyspark.sql import SparkSession
from SparkDBUtils import SparkDB
import delta
import datetime as dt
import pyspark.sql
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StructType, StructField, IntegerType, TimestampType

sparkdb = SparkDB()
spark = sparkdb.spark

In [50]:
df = spark.table("producto_dim")
df.write.saveAsTable("producto_dim2","delta","overwrite")

In [15]:
df2 = spark.table("producto_dim2")
df2.select("product_id").distinct().show()

In [17]:
df.summary().show()

+-------+----------------+--------------------+--------------+--------------------+------------------+-----------------+---------------+
|summary|     id_producto|             product|         brand|          categories|        product_id|        categoria|          units|
+-------+----------------+--------------------+--------------+--------------------+------------------+-----------------+---------------+
|  count|            6921|                6921|          6683|                6921|              6921|             6921|           6921|
|   mean|          3461.0|                null|          null|                null|185929.78613976465|             null|           null|
| stddev|1998.06493888462|                null|          null|                null| 93654.91515911062|             null|           null|
|    min|               1|1902 licor de gin...|ABADIA DA COVA|['Bebidas', 'Aguas']|               100|          Bebidas|          €/Kg.|
|    25%|            1730|               

In [22]:
df3 = df2.where("product_id < 117788")

In [51]:
a = delta.DeltaTable.forName(spark, "producto_dim2")
type(a)

delta.tables.DeltaTable

In [67]:
cols = ["id"]
data = [1,100]

dfb = spark.createDataFrame(data, pyspark.sql.types.IntegerType()).toDF(*cols)

In [74]:
l = list(dfb.select("id").toPandas()["id"])
",".join(l)

TypeError: sequence item 0: expected str instance, int found

In [52]:
a.toDF().show()

+-----------+--------------------+----------------+--------------------+----------+----------+-----------------+-----+--------------------+
|id_producto|             product|           brand|          categories|product_id|      date|        categoria|units|             ts_load|
+-----------+--------------------+----------------+--------------------+----------+----------+-----------------+-----+--------------------+
|          1|CARBONELL aceite ...|       CARBONELL|['Despensa', 'Ace...|       100|2022-11-21|         Despensa| €/l.|2023-02-26 19:35:...|
|          2|YBARRA salsa cock...|          YBARRA|['Despensa', 'Sal...|      1000|2022-11-21|         Despensa| €/l.|2023-02-26 19:35:...|
|          3|LU PRINCIPE Estre...|     LU PRINCIPE|['Despensa', 'Des...|    100267|2022-11-21|         Despensa|€/Kg.|2023-02-26 19:35:...|
|          4|LU PRINCIPE Estre...|     LU PRINCIPE|['Despensa', 'Des...|    100268|2022-11-21|         Despensa|€/Kg.|2023-02-26 19:35:...|
|          5|GULLON 

In [9]:
spark.sql("show tables in default").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|            date_dim|      false|
|  default|precio_dia_agg_no...|      false|
|  default|   producto_dia_fact|      false|
|  default|        producto_dim|      false|
|  default|       producto_dim2|      false|
|  default|       sequences_cfg|      false|
+---------+--------------------+-----------+



In [6]:
spark.sql("select * from sequences_cfg").show()

+------------+----+--------------------+
|  table_name|  id|             ts_load|
+------------+----+--------------------+
|producto_dim|6871|2023-02-26 19:32:...|
|    date_dim|  39|2023-02-26 19:32:...|
+------------+----+--------------------+



In [7]:
spark.sql("select * from producto_dia_fact").count()

224904

In [8]:
spark.sql("select * from precio_dia_agg_norm_fact").count()

39

In [3]:
producto_dia_fact = spark.table("producto_dia_fact").alias("pf")
date_dim = spark.table("date_dim").alias("d")
producto_dim = spark.table("producto_dim").alias("p")

In [4]:
p = producto_dia_fact\
    .join(date_dim, "id_date")\
    .join(producto_dim, "id_producto")

subset = p.select(f.column("pf.price"),
        f.column("d.date"),
        f.column("p.product"))\
    .where("p.id_producto == 4846")

subset.show(10)

+-----------------+----------+--------------------+
|            price|      date|             product|
+-----------------+----------+--------------------+
|7.989999771118164|2022-11-21|1902 licor de gin...|
|7.989999771118164|2022-12-08|1902 licor de gin...|
|7.989999771118164|2022-12-06|1902 licor de gin...|
|7.989999771118164|2022-12-29|1902 licor de gin...|
|7.989999771118164|2023-01-16|1902 licor de gin...|
|7.989999771118164|2023-01-12|1902 licor de gin...|
|7.989999771118164|2023-01-13|1902 licor de gin...|
|7.989999771118164|2022-11-26|1902 licor de gin...|
|7.989999771118164|2022-12-01|1902 licor de gin...|
|7.989999771118164|2022-11-23|1902 licor de gin...|
+-----------------+----------+--------------------+
only showing top 10 rows



In [35]:
pan = subset.toPandas()

pan.to_csv("c:/tmp/extract/test.csv", index=False)

In [15]:
subset.write\
    .option("header", "true")\
    .csv("file:///c:/tmp/extract/test.csv")

In [39]:
spark.table("date_dim").toPandas().to_csv("c:/tmp/extract/date_dim.csv", index=False)

c:\users\carlos\proyectos\foodecommercescraper\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
